In [10]:
import torch

In [11]:
samples = torch.tensor([1, 0, 1, 0, 1], [0, 1, 0, 1, 0])
num_sample, num_visible_node = samples.shape
samples_array = []
for copy_index in range(2):
    rolled_samples = torch.roll(samples, shifts=copy_index, dims=0)
    samples_array.append(rolled_samples)
samples_array = torch.hstack(samples_array)
samples_array

TypeError: tensor() takes 1 positional argument but 2 were given

In [9]:
swapped_samples_array = torch.roll(samples_array, shifts = 2, dims=1)
swapped_samples_array

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)